# Modeling

In [1]:
import gzip
import pickle
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
plt.style.use('ggplot')

DATA_PATH = '/root/Workspace/DataWarehouse/stMary_RRpo'

with gzip.open(f'{DATA_PATH}/21_230518_resamp_sliced125_filt_patient_stmary.pickle.gzip', 'rb') as f:
    dataset = pickle.load(f)

print(len(dataset), len(dataset[0][0]))

random.seed(42)
random.shuffle(dataset)

pleths = []
resps = []
for ppg, rr in dataset:
    pleths.append(ppg.astype(np.float64))
    resps.append(rr)

pleths = np.asarray(pleths)
resps = np.asarray(resps)
print(pleths.shape, resps.shape)

scaler = MinMaxScaler()
scaled_pleths = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in pleths])
print(scaled_pleths.shape, type(scaled_pleths[0][0][0]))

ratio_tr = 0.8
train_x, train_y = scaled_pleths[:int(len(scaled_pleths)*ratio_tr)], resps[:int(len(resps)*ratio_tr)]
val_x, val_y = scaled_pleths[int(len(scaled_pleths)*ratio_tr):], resps[int(len(resps)*ratio_tr):]
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)

6508 1800
(6508, 1800) (6508,)
(6508, 1800, 1) <class 'numpy.float64'>
(5206, 1800, 1) (5206,)
(1302, 1800, 1) (1302,)


## Architecture: LSTM Family
LSTM 또한 어느정도 유의미성을 보일 것으로 기대되므로 시도해보고자 한다.
- Vanilla LSTM
- Conv-LSTM
- BiLSTM

In [32]:
import os
import keras
import tensorflow as tf
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import LSTM, Flatten, Dense, Conv1D, Bidirectional, Concatenate, Dropout, ConvLSTM1D, AveragePooling1D, BatchNormalization, Activation
print(f'Is GPU Avaliable: {tf.config.list_physical_devices("GPU")}')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
class VanillaLSTM(Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.lstm1 = LSTM(15)
        self.d1 = Dense(1)
    
    def call(self, inputs, *args, **kwargs):
        x = inputs
        x = self.lstm1(x)
        x = Flatten()(x)
        return self.d1(x)

In [38]:
class ConvLSTM(Model):
    def __init__(self):
        super(ConvLSTM, self).__init__()
        self.conv1 = Conv1D(filters=64, kernel_size=3, strides=1, padding='valid')
        self.conv2 = Conv1D(filters=128, kernel_size=3, strides=1, padding='valid')
        self.lstm1 = LSTM(units=10, activation='relu', return_sequences=True)
        self.dense1 = Dense(100, activation='relu')
        self.dense2 = Dense(20, activation='relu')
        self.dense3 = Dense(1)
        self.bn1 = BatchNormalization()
        self.bn2 = BatchNormalization()


    def call(self, inputs, training=None, mask=None):
        x = inputs
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = Activation('relu')(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.lstm1(x)
        x = Flatten()(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x
    

    @tf.function
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}
    
    @tf.function
    def test_step(self, data):
        x, y = data

        y_pred = self(x, training=False)
        self.compiled_loss(y, y_pred)
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

In [5]:
class BahdanauAttention(Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)


    def call(self, values, query, training=None, mask=None):
        hidden_with_time_axis = tf.expand_dims(query, 1)

        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)
        ))

        attention_weights = tf.nn.softmax(score, axis=1)

        context = attention_weights * values
        context = tf.reduce_sum(context, axis=1)

        return context, attention_weights

In [6]:
class LSTMAttention(Model):
    def __init__(self, units, units_attn, dropout):
        super(LSTMAttention, self).__init__()
        self.bilstm1 = Bidirectional(LSTM(units=units, dropout=dropout, return_sequences=True))
        self.bilstm2 = Bidirectional(LSTM(units=units, dropout=dropout, return_sequences=True, return_state=True))
        self.attention = BahdanauAttention(units_attn)
        self.d20 = Dense(20, activation='relu')
        self.d1 = Dense(1)

    @tf.function
    def call(self, inputs, training=None, mask=None):
        x = self.bilstm1(inputs)
        x, forward_h, _, backward_h, _ = self.bilstm2(x)
        state_h = Concatenate()([forward_h, backward_h])
        context, attention_weights = self.attention(x, state_h)
        x = self.d20(context)
        x = Dropout(0.5)(x)
        x = self.d1(x)
        return x

In [7]:
# lstm = Bidirectional(LSTM(10, dropout=0.5, return_sequences=True))(train_x)
# print(lstm.shape)
# lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(10, dropout=0.5, return_sequences=True, return_state=True))(lstm)
# print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)
# # lstm 전체의 히든 스테이트, 순방향 hidden, 순방향 cell, 역방향 hidden, 역방향 cell
# state_h = Concatenate()([forward_h, backward_h]) # 양방향 마지막 h
# state_c = Concatenate()([forward_c, backward_c]) # 양방향 마지막 c
# print(state_h.shape, state_c.shape)
# attention = BahdanauAttention(64)
# context, attention_weights = attention(lstm, state_h)
# print(context.shape, attention_weights.shape)
# dense1 = Dense(20, activation='relu')(context)
# dropout = Dropout(0.5)(dense1)
# output = Dense(1)(dropout)

In [34]:
EPOCHS = 1000
BATCH_SIZE = 256
LR = 0.001
kf = KFold(n_splits=5)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    # ModelCheckpoint('../models/230522-Resnet', monitor='val_loss', save_best_only=True)
]

In [39]:
# model1 = VanillaLSTM()
# model2 = LSTMAttention(20, 64, 0.5)
model3 = ConvLSTM()

In [40]:
model3.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    # optimizer=tf.keras.optimizers.SGD(learning_rate=LR, momentum=0.9, weight_decay=0.0001),
    loss=keras.losses.MeanAbsoluteError(),
    metrics=keras.metrics.MeanAbsoluteError()
)

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(BATCH_SIZE)

In [41]:
with tf.device('/GPU:0'):
    history = model3.fit(
        train_dataset,
        epochs=EPOCHS,
        callbacks=callbacks,
        validation_data=val_dataset
    )

Epoch 1/1000


2023-05-25 17:55:47.332746: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5206]
	 [[{{node Placeholder/_1}}]]


21/21 [==============================] - ETA: 0s - loss: 7.4189 - mean_absolute_error: 7.4189

2023-05-25 17:56:36.961013: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1302]
	 [[{{node Placeholder/_1}}]]


21/21 [==============================] - 51s 2s/step - loss: 7.4189 - mean_absolute_error: 7.4189 - val_loss: 16.4788 - val_mean_absolute_error: 16.4788 - lr: 0.0010
Epoch 2/1000
21/21 [==============================] - 49s 2s/step - loss: 4.0801 - mean_absolute_error: 4.0801 - val_loss: 14.2745 - val_mean_absolute_error: 14.2745 - lr: 0.0010
Epoch 3/1000
21/21 [==============================] - 46s 2s/step - loss: 3.9412 - mean_absolute_error: 3.9412 - val_loss: 10.9348 - val_mean_absolute_error: 10.9348 - lr: 0.0010
Epoch 4/1000
21/21 [==============================] - 48s 2s/step - loss: 3.9823 - mean_absolute_error: 3.9823 - val_loss: 8.9017 - val_mean_absolute_error: 8.9017 - lr: 0.0010
Epoch 5/1000
21/21 [==============================] - 47s 2s/step - loss: 3.6961 - mean_absolute_error: 3.6961 - val_loss: 7.5749 - val_mean_absolute_error: 7.5749 - lr: 0.0010
Epoch 6/1000
21/21 [==============================] - 46s 2s/step - loss: 3.4683 - mean_absolute_error: 3.4683 - val_loss:

KeyboardInterrupt: 

## Load model

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(128)
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(128)

model = keras.models.load_model('../models/230525-VanillaLSTM')

2023-05-25 16:31:50.657273: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14432 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:73:00.0, compute capability: 7.5
2023-05-25 16:31:51.270887: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-05-25 16:31:51.271032: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_g

In [5]:
pred_y = model.predict(train_dataset)
print(pred_y.shape, train_y.shape)
abs_err = abs(train_y.reshape(-1,1) - pred_y)
print(f'{np.mean(abs_err)} ± {np.std(abs_err)}')

2023-05-25 16:31:52.095008: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5206]
	 [[{{node Placeholder/_1}}]]
2023-05-25 16:31:52.153957: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1800,1]
	 [[{{node inputs}}]]


 1/41 [..............................] - ETA: 43s

2023-05-25 16:31:52.969827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700


41/41 [==============================] - 2s 27ms/step
(5206, 1) (5206,)
4.512258993493929 ± 3.3242894680147885


In [6]:
pred_y = model.predict(val_dataset)
print(pred_y.shape, val_y.shape)
abs_err = abs(val_y.reshape(-1,1) - pred_y)
print(f'{np.mean(abs_err)} ± {np.std(abs_err)}')

 5/11 [============>.................] - ETA: 0s

2023-05-25 16:31:56.918444: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1302]
	 [[{{node Placeholder/_1}}]]


11/11 [==============================] - 0s 28ms/step
(1302, 1) (1302,)
4.572838472697409 ± 3.334458200936277
